In [11]:
object_type = 'charcoal_hearth_hill'
object_annots_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\charcoalhearths_data\\'+object_type+'\\annots\\'
object_images_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\charcoalhearths_data\\'+object_type+'\\images\\'
object_boxes_output_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\charcoalhearths_data\\'+object_type+'\\selected_training\\'

import cv2
from xml.etree import ElementTree
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(object_annots_dir) if isfile(join(object_annots_dir, f))]

for fn in onlyfiles:
    print(fn)
    tree = ElementTree.parse(object_annots_dir+fn)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    
    fn_image = root.find('./filename').text
    print(fn_image)
    box_num=0
    for box in root.findall('.//bndbox'):
        box_num=box_num+1
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        if(ymin>ymax):
            ytemp = ymin
            ymin = ymax
            ymax=ytemp
        if(xmin>xmax):
            xtemp = xmin
            xmin = xmax
            xmax=xtemp            
        coors = [xmin, ymin, xmax, ymax]
        #print(coors)
        
        img = cv2.imread(object_images_dir+fn_image)
        crop_img = img[ymin:ymax, xmin:xmax]
        
        print(crop_img.shape)
        fntd = object_boxes_output_dir+"td_"+fn[:5]+"_b"+str(box_num)+fn_image[5:]
        print(fntd)
        cv2.imwrite(fntd,crop_img)

BlueMtHearthforShawnJeff.xml
BlueMtHearthforShawnJeff.jpg
(40, 28, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b1tHearthforShawnJeff.jpg
(39, 29, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b2tHearthforShawnJeff.jpg
(39, 28, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b3tHearthforShawnJeff.jpg
(40, 28, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b4tHearthforShawnJeff.jpg
(39, 29, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b5tHearthforShawnJeff.jpg
(40, 29, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b6tHearthforShawnJeff.jpg
(33, 26, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b7tHearth

In [13]:
#print(construction_type)
# display image with masks and bounding boxes
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes

# train set
train_set = ObjectDataset()

object_type = 'charcoal_hearth_hill'
object_annots_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\charcoalhearths_data\\'+object_type

train_set.load_dataset(object_annots_dir, is_train=True)
train_set.prepare()
# define image id
image_id = 0
# load the image
image = train_set.load_image(image_id)
# load the masks and the class ids
mask, class_ids = train_set.load_mask(image_id)
# extract bounding boxes from the masks
bbox = extract_bboxes(mask)
# display image with masks and bounding boxes
display_instances(image, bbox, mask, class_ids, train_set.class_names)

ModuleNotFoundError: No module named 'mrcnn'